# pre_processing

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nbimporter
import feature_generation

Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from pre_processing.ipynb


In [3]:
features = feature_generation.get_features()
features_types = feature_generation.get_features_types()

In [4]:
def load_appended_dataset():
    """Carga ambos dataframes (train y test) y devuelve un solo dataframe que es el resultado de realizar un
    append entre ambos. El dataset de test esta al final."""

    train = pd.read_csv('./data/train.csv', index_col='id', parse_dates=['fecha'])
    test = pd.read_csv('./data/test.csv', index_col='id', parse_dates=['fecha'])
    df = train.append(test, sort=False)
    df = df[['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
         'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
         'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio']]
    
    whitelist_tipodepropiedad = df['tipodepropiedad'].value_counts().index.to_list()[:9]
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: x if x in whitelist_tipodepropiedad else 'Otro') 
    
    #whitelist_ciudad = df['ciudad'].value_counts().index.to_list()[:80]
    #df['ciudad'] = df['ciudad'].apply(lambda x: x if x in whitelist_ciudad else 'Otro')
    
    categoricas = ['tipodepropiedad', 'ciudad', 'provincia']
    df[categoricas] = df[categoricas].fillna('unknown')
    df[categoricas] = df[categoricas].astype('category')
    enteras = ['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']
    df[enteras] = df[enteras].astype('int8')
    
    return df

In [5]:
def separate_train_test(df):
    """Separa df compuesto por sus primeras 240.000 filas de train.csv y sus 60.000 ultimas filas de test
    en los dos dataframes correspondientes."""
    
    train = df.head(240000)
    test = df.tail(60000).drop('precio', axis=1)
    return train, test

In [6]:
def load_datasets():
    """Carga ambos dataframes (train y test) y devuelve dos datasets en el formato:
    train_df, test_df"""
    
    df = load_appended_dataset()
    return separate_train_test(df)

In [7]:
def load_featured_appended_dataset():
    """Carga el .csv con los features generados, y lo formatea para ahorrar memoria. Devuelve un unico
    dataframe cuyas primeras 240.000 filas son de train.csv y sus ultimas 60.000 de test."""
    features_types = feature_generation.get_features_types()
    
    df = pd.read_csv('./data/full_featured.csv', index_col='id')
    df[features_types['bernoulli']] = df[features_types['bernoulli']].astype('uint8')
    df[features_types['float']] = df[features_types['float']].astype('float')
    df[features_types['uint8']] = df[features_types['uint8']].astype('uint8')
    df[features_types['uint16']] = df[features_types['uint16']].astype('uint16')
    df[features_types['uint32']] = df[features_types['uint32']].astype('uint32')
    
    return df

In [8]:
def load_featured_datasets():
    """Devuelve dos datasets en el formato:
    train_df, test_df
    Para los datasets con las features generadas."""
    
    df = load_featured_appended_dataset()
    return separate_train_test(df)

# Pre procesamiento de datasets externos
### Cotizacion USD

In [44]:
usd = pd.read_csv('data/usd_mxn_diario.csv')

#### funciones aux

In [45]:
def aniomes(x):
    anio=x[6:]
    mes=x[3:5]
    return anio+mes

def numeric(x):
    entero, fraccion = x.split(',')
    return int(entero) + int(fraccion)/10000

def varianza(x):
    x = x.rstrip('%')
    entero, fraccion = x.split(',')
    num = entero+"."+fraccion
    return float(num)

In [46]:
usd['aniomes'] = usd['Fecha'].map(aniomes)

In [47]:
valores = ['Último', 'Apertura', 'Máximo', 'Mínimo']
for valor in valores:
    usd[valor] = usd[valor].map(numeric)
    
usd['usd_varianza'] = usd['% var.'].map(varianza)
usd['daily_mean'] = usd.apply(lambda x: (x['Último'] + x['Apertura'])/2, axis=1)

In [48]:
usd = usd[['aniomes', 'usd_varianza', 'daily_mean']]

In [49]:
usd = usd.groupby('aniomes')['usd_varianza', 'daily_mean'].agg({'usd_varianza':'sum', 'daily_mean':'mean'})

In [50]:
usd.head()

,usd_varianza,daily_mean
aniomes,,
201201,-6.71,13.419316
201202,-1.50,12.789329
201203,-0.27,12.734559
201204,1.67,13.047955
201205,9.99,13.630126


In [51]:
usd.to_csv('data/usd_mxn_featured.csv')